In [2]:
class node:

    def __init__(self, funct, right, left):
        self.funct = funct
        self.left = left
        self.right = right

    def __str__(self):
        return f"function: {self.funct},right_function : {self.right} left_function : {self.left}"


In [3]:
def cofactor(f,cwt):
    # seperating the string for further processing
    f = f.split("+")
    f = [list(i) for i in f]

    # Function for finding the cofactor of a boolean SoP expression with respect to a single literal
    def cofactor_1(f, cwt):

        for i in range(len(f)):
            if cwt.islower():
                f[i] = [j.replace(cwt.lower(), "1") for j in f[i]]
                f[i] = [j.replace(cwt.upper(), "0") for j in f[i]]
            else:
                f[i] = [j.replace(cwt.lower(), "0") for j in f[i]]
                f[i] = [j.replace(cwt.upper(), "1") for j in f[i]]
            # f[:] is used to make a copy of f
        for i in f[:]:
            rem = False

            for j in i:
                if j == '0':
                    rem = True
                    break
            if rem:
                f.remove(i)

            for j in i:
                if j == '1':
                    i.remove(j)
                    if len(i) == 0:
                        i.append("1")

        for i in f:
            if i == ['1']:
                return i
        if len(f) == 0:
            return '0'
        return f
    
    # cwt is split to list and iterated over each literal
    cwt = list(cwt)
    for i in cwt:
        f = cofactor_1(f, i)

    # Combining the reduced expression to a single string
    reduced_exp = ""
    for i in f:
        reduced_exp += "".join(i)
        reduced_exp += "+"
    reduced_exp = reduced_exp.rstrip("+")

    return reduced_exp

In [4]:
def AND(f1, f2):

    f1 = f1.split("+")
    f1 = [list(i) for i in f1]

    f2 = f2.split("+")
    f2 = [list(i) for i in f2]

    F = []

    for i in f1:
        for j in f2:
            F.append(i+j)
    
    # for removing duplicate literals in a cube
    for i in range(len(F)):
        F[i] = [x for x in set(F[i])]

    # for removing duplicate cubes
    F = [list(x) for x in set(map(tuple, F))]

    # replace and of a literal and its compliment with 0
    for i in range(len(F)):
        if any(j.isupper() and j.lower() in F[i] for j in F[i]):
            F[i] = ['0']

    for i in F[:]:
        rem = False

        # remove term if it is 0
        for j in i:
            if j == '0':
                rem = True
                break
        if rem:
            F.remove(i)
            if len(F) == 0:
                F.append("0")

        # remove multiple 1's
        for j in i:
            if j == '1':
                i.remove(j)
                if len(i) == 0:
                    i.append("1")
    
    for i in F:
        if i == ['1']:
            F = i

    f = ""
    for i in F:
        f += "".join(i)
        f += "+"
    f = f.rstrip("+")

    return f

In [5]:
def NOT(f):
    if f == '1':
        return '0'
    if f == '0':
        return '1'
    # seperating the string for further processing
    f = f.split("+")
    f = [list(i) for i in f]

    # print(f)

    for i in range(len(f)):
        for j in range(len(f[i])):
            c = f[i][j]
            f[i][j] = c.upper() if c.islower() else c.lower()

    # print(f)
    for i in range(len(f)):
        str = []
        for j in range(len(f[i])):
            str += f[i][j] + ("+" if j != len(f[i])-1 else "") 
        f[i] = str

    # print(f)
    str1 = ""
    for j in range(len(f[0])):
        str1+=f[0][j]
    F = str1
    
    for i in range(1,len(f)):
        str2 = ""
        for j in range(len(f[i])):
            str2+=f[i][j]

        # print(str1+'\n')
        # print(str2+'\n')
        
        F = AND(F, str2)
        # f[i] = AND(str1, str2)

    return F

In [6]:
def is_same(a,b):
    
    if a.left == b.left and a.right == b.right:
        return True
        
    return False

In [7]:
# unique_table = []

# def to_unique_table(a):
#     for i in unique_table:
#         if is_same(a,i):
#             return
#     unique_table.append(a)

# def build(node,order):
#      if node.funct == '0':
#         return zero_node
#     elif node.funct == '1':
#         return one_node
#     else:
#         var = order[0]
        

#         right_funct = cofactor(node.funct, var)
#         left_funct = cofactor(node.funct, NOT(var))

#         right_node = build(node(right_funct, cofactor(right_funct,order[1]), cofactor(right_funct,NOT(order[1]))),order[1:])
#         left_node = build(node(left_funct, cofactor(left_funct,order[1]), cofactor(left_funct,NOT(order[1]))),order[1:])



#         if is_same(right_node, left_node):
#             to_unique_table(right_node)
#         else:
#             to_unique_table(right_node)
#             to_unique_table(left_node)

In [11]:
unique_table = []
def to_unique_table(a):
    for i in unique_table:
        if is_same(a,i):
            return a
    unique_table.append(a)
    return a

def build(function,order,i):
    if function == '0':
        return to_unique_table(node('0', '0', '0'))
    elif function == '1':
        return to_unique_table(node('1', '1', '1'))
    else:
        var = order[i]

        right_funct = cofactor(function, var)
        # print(right_funct)
        left_funct = cofactor(function, NOT(var))
        # print(left_funct)
        i+=1
        right_build = build(right_funct,order,i)
        # print(right_build.funct)
        left_build = build(left_funct,order,i)

        if is_same(right_build, left_build):
            # print(is_same(right_build, left_build),right_build,left_build)
            return to_unique_table(right_build)
        else:
            # print(node(function, right_funct, left_funct))
            return to_unique_table(node(function, right_funct, left_funct))

In [25]:
# build('a+b',['b','a'],0)
unique_table = []
build('ab+ca+dcb',['b','d','a','c'],0)

print(f"{[unique_table[i].funct for i in range(len(unique_table))]},{[unique_table[i].right for i in range(len(unique_table))]},{[unique_table[i].left for i in range(len(unique_table))]}")
# unique_table

['1', '0', 'c', 'a+ca+c', 'a+ca+dc', 'ca', 'ab+ca+dcb'],['1', '0', '1', '1', 'a+ca+c', 'c', 'a+ca+dc'],['1', '0', '0', 'c', 'a+ca', '0', 'ca']


In [26]:
ROBDD = { unique_table[i].funct : [unique_table[i].left,unique_table[i].right] for i in range(len(unique_table))}
ROBDD = dict(list(ROBDD.items())[::-1])
print(ROBDD)

{'ab+ca+dcb': ['ca', 'a+ca+dc'], 'ca': ['0', 'c'], 'a+ca+dc': ['a+ca', 'a+ca+c'], 'a+ca+c': ['c', '1'], 'c': ['0', '1'], '0': ['0', '0'], '1': ['1', '1']}


In [20]:
a = ['a','A']

a = [i.lower() for i in a]
a

['a', 'a']